In [1]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np

#load xgb model
xgb_model = XGBClassifier()
xgb_model.load_model(r"C:\Users\miles\favela_analysis\output\morro_da_guaiba\morro_da_guaiba_xgb_model.json")


In [ ]:
# reconstruct label_encoder
loaded_classes = np.load(r"C:\Users\miles\favela_analysis\output\morro_da_guaiba\morro_da_guaiba_label_encoder_classes.npy")

# Recreate the LabelEncoder and assign the loaded classes_
label_encoder = LabelEncoder()
label_encoder.classes_ = loaded_classes  # For correct class mapping


In [3]:
# load neccessary data
with open(r"C:\Users\miles\favela_analysis\output\morro_da_guaiba\morro_da_guaiba_X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

y_test = np.load(r"C:\Users\miles\favela_analysis\output\morro_da_guaiba\morro_da_guaiba_y_test.npy")


In [5]:
y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
class_report_dict = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)

print("Accuracy: ", accuracy)
print(class_report_dict)


Accuracy:  94.62285287528006
{'favela': {'precision': 0.9567966280295047, 'recall': 0.966986155484558, 'f1-score': 0.961864406779661, 'support': 939.0}, 'not-favela': {'precision': 0.9205128205128205, 'recall': 0.8975, 'f1-score': 0.9088607594936708, 'support': 400.0}, 'accuracy': 0.9462285287528006, 'macro avg': {'precision': 0.9386547242711626, 'recall': 0.932243077742279, 'f1-score': 0.9353625831366659, 'support': 1339.0}, 'weighted avg': {'precision': 0.9459575518482696, 'recall': 0.9462285287528006, 'f1-score': 0.9460306062461314, 'support': 1339.0}}
